In [6]:
import os
import sys
os.chdir('/home/peitian_zhang/Codes/NR')
sys.path.append('/home/peitian_zhang/Codes/NR')

import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import GloVe
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from utils.MIND import MIND_iter,MIND_map
from utils.utils import getVocab,getLoss,constructBasicDict,run_eval,run_train
from models.FIM import FIMModel

## setting up the *FIM* model

### define paths and hyperparameters, load data

all these hyper parameters are fixed according to the paper [\[29\] Fine-grained Interest Matching for Neural News Recommendation](https://www.aclweb.org/anthology/2020.acl-main.77.pdf)

- *mode*: data to read (*demo*/*small*/*large*)

- *batch_size*: size of each minibatch

- *title_size*: max word capacity of title

- *his_size*: max record capacity of click history

- *npratio*: number of negtive sampling

- *dilation_level*: levels of diferrent dilation rate

- *kernel_size*: size of 1dCNN kernel

- *filter_num*: number of kernels in 1D CNN, which is also embedding dimension of news/user

- *embedding_dim*: word embedding dimension

- *metrics*: metrics to be used in evaluating

- *gpu*: gpu ID if available

- *attrs*: the columns used to construct vocab

### Note 

Although I list *dilation_level* and *kernel_size*, some properties related to both variables are fixed in the FIM model because I don't have time to well-design my model to make it adapt to these variables dynamically. 

In [2]:
hparams = {
    'mode':'demo',
    'name':'fim',
    'epochs':10,
    'npratio':4,
    'batch_size':5,
    'title_size':18,
    'his_size':50,
    'kernel_size':3,
    'dilation_level':3,
    'filter_num':150,
    'embedding_dim':300,
    'dropout_p':0.1,
    'metrics':'group_auc,ndcg@5,ndcg@10,mean_mrr',
    'device':'cuda:0',
    'attrs': ['title','category','subcategory']
}

# customize your path here

news_file_train = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_train/news.tsv'
news_file_test = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_dev/news.tsv'
news_file_pair = (news_file_train,news_file_test)

behavior_file_train = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_train/behaviors.tsv'
behavior_file_test = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_dev/behaviors.tsv'
behavior_file_pair = (behavior_file_train,behavior_file_test)

save_path = 'models/model_params/{}_{}_{}'.format(hparams['name'],hparams['mode'],hparams['epochs']) +'.model'

if not os.path.exists('data/dictionaries/vocab_{}_{}.pkl'.format(hparams['mode'],'_'.join(hparams['attrs']))):
    constructBasicDict(news_file_pair,behavior_file_pair,hparams['mode'],hparams['attrs'])

device = torch.device(hparams['device']) if torch.cuda.is_available() else torch.device("cpu")

dataset_train = MIND_map(hparams=hparams,news_file=news_file_train,behaviors_file=behavior_file_train)

dataset_test = MIND_iter(hparams=hparams,news_file=news_file_test,behaviors_file=behavior_file_test, mode='test')

vocab = dataset_train.vocab
embedding = GloVe(dim=300,cache='.vector_cache')
vocab.load_vectors(embedding)

loader_train = DataLoader(dataset_train,batch_size=hparams['batch_size'],shuffle=True,pin_memory=True,num_workers=3,drop_last=True)
loader_test = DataLoader(dataset_test,batch_size=hparams['batch_size'],pin_memory=True,num_workers=0,drop_last=True)

In [9]:
# you can load model or train

# fimModel = FIMModel(vocab=vocab,hparams=hparams).to(device)
# fimModel.load_state_dict(torch.load(save_path))
# fimModel.train()
from datetime import datetime
fimModel = FIMModel(vocab=vocab,hparams=hparams).to(device)
fimModel.train()
writer = SummaryWriter('data/tb/{}/{}/{}/'.format(hparams['name'], hparams['mode'], datetime.now().strftime("%Y%m%d-%H")))

### train the model

In [10]:
if fimModel.training:
    print("training...")
    loss_func = getLoss(fimModel)
    optimizer = optim.Adam(fimModel.parameters(),lr=0.001)
    fimModel = run_train(fimModel,loader_train,optimizer,loss_func,writer=writer,epochs=10, interval=15)

training...
epoch 0 , step 390 , loss: 1.5386: : 400it [00:22, 17.99it/s]
epoch 1 , step 390 , loss: 1.4396: : 400it [00:20, 19.69it/s]
epoch 2 , step 390 , loss: 1.3263: : 400it [00:20, 19.88it/s]
epoch 3 , step 390 , loss: 1.1989: : 400it [00:20, 19.67it/s]
epoch 4 , step 390 , loss: 1.0289: : 400it [00:20, 19.63it/s]
epoch 5 , step 390 , loss: 0.8166: : 400it [00:20, 19.93it/s]
epoch 6 , step 390 , loss: 0.5484: : 400it [00:20, 19.90it/s]
epoch 7 , step 390 , loss: 0.3282: : 400it [00:20, 19.99it/s]
epoch 8 , step 390 , loss: 0.2571: : 400it [00:20, 19.93it/s]
epoch 9 , step 390 , loss: 0.1986: : 400it [00:20, 19.92it/s]


### test & evaluate

In [ ]:
fimModel.eval()
fimModel.cdd_size = 1

run_eval(fimModel,loader_test)

### save the model

In [ ]:
fimModel.cdd_size = 5
torch.save(fimModel.state_dict(), save_path)